In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Load lenti data for all cell types (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'lenti')

In [ ]:
ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata['ts_label'] = metadata['ts_kind'].replace(ts_label)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

# Create palettes for application transfections
metadata2 = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'applications')
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
app_palette = metadata_dict2['color']
app_markers = metadata_dict2['markers']

In [ ]:
# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df = data[(data['construct'].isin(['UI','UT'])) | (data['virus_dilution']==0)].groupby(['cell','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6, 1+1.5*4))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 6, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[1]+[1.5]*4, width_ratios=[1]*6)

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:]),
    'B': fig.add_subfigure(fig_gridspec[1,:3]),
    'C': fig.add_subfigure(fig_gridspec[1,3:]),
    'D': fig.add_subfigure(fig_gridspec[2,:3]),
    'E': fig.add_subfigure(fig_gridspec[2,3:]),
    'F': fig.add_subfigure(fig_gridspec[3,:3]),
    'G': fig.add_subfigure(fig_gridspec[3,3:]),
    'H': fig.add_subfigure(fig_gridspec[4,:2]),
    'I': fig.add_subfigure(fig_gridspec[4,2:4]),
    'I2': fig.add_subfigure(fig_gridspec[4,4:]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_applications'/'fig_applications.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = quantiles[(quantiles['dox']==1000) & (quantiles['moi']==5) & (quantiles['cell']=='neuron')].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker',)
sns.despine(ax=ax)

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = quantiles[(quantiles['group'].isin(['controller','base'])) & (quantiles['dox']==1000) & 
                    (quantiles['moi']==1) & (quantiles['cell']=='MEF') & (quantiles['design']<=1) &
                    (((quantiles['ts_kind']=='na') & (quantiles['biorep']==6)) | ((quantiles['biorep']==1)))
                    ].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(1e3,2e5))
sns.despine(ax=ax)
baseline = baseline_df.loc[baseline_df['cell']=='MEF', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = quantiles[(quantiles['group'].isin(['controller','base'])) & (quantiles['dox']==1000) & 
                    (quantiles['moi']==1) & (quantiles['cell']=='tcell') & (quantiles['biorep']==1)].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker',)
sns.despine(ax=ax)
ax.yaxis.set_tick_params(which='minor', left=False)
baseline = baseline_df.loc[baseline_df['cell']=='tcell', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = quantiles[(quantiles['group'].isin(['controller','base'])) & (quantiles['cell']=='iPS11') &
                    (quantiles['biorep']==3) & (quantiles['dox']==1000)].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker',)
sns.despine(ax=ax)
ax.xaxis.set_minor_formatter('')
ax.yaxis.set_tick_params(which='minor', left=False)

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['I']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.35, top=0.3, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = quantiles[(quantiles['cell']=='therapeutic') & (quantiles['dox']==1000) & 
                    (quantiles['name'].str.contains('FXN')) & (quantiles['biorep']==1)].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=app_palette, 
             legend=False, dashes=False, style='construct', markers=app_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker', title='FXN')
sns.despine(ax=ax)
baseline = baseline_df.loc[baseline_df['cell']=='therapeutic', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.yaxis.set_tick_params(which='minor', left=False)

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=app_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['I2']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.35, top=0.3, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = quantiles[(quantiles['cell']=='therapeutic') & (quantiles['dox']==1000) & 
                    (quantiles['name'].str.contains('FMRP')) & (quantiles['biorep']==1)].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=app_palette, 
             legend=False, dashes=False, style='construct', markers=app_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker', title='FMRP1')
sns.despine(ax=ax)
baseline = baseline_df.loc[baseline_df['cell']=='therapeutic', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=app_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.45, top=0.3, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1.75), wspace=0.5))

# slope
ax = axes[0]
plot_df = stats[(stats['cell']=='neuron') & (stats['moi']==1)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='slope', hue='construct', palette=app_palette,
                  legend=False, ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0,1.2))
ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], ha='right', rotation=45)
sns.despine(ax=ax)

# CV
ax = axes[1]
plot_df = stats[(stats['cell']=='neuron') & (stats['moi']==1)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_variation', hue='construct', palette=app_palette,
                  legend=False, ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='', ylim=(0.3,1))
ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], ha='right', rotation=45)
sns.despine(ax=ax)

# CDF
ax = axes[2]
plot_df = quantiles[(quantiles['cell']=='neuron') & (quantiles['moi']==1)]

sns.kdeplot(data=plot_df, x='output', hue='construct', palette=main_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(2e1,2e4), title='CDF')
sns.despine(ax=ax)
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_applications.pdf'))